In [1]:
import os

import io

import csv 

from zipfile import ZipFile

import glob 

import random 

from google.cloud import bigquery

from google.oauth2 import service_account

import sqlite3

In [2]:
# Authentication: these keys will be different on your device, must change to run your own code. 
# Path to connect to GBQ

service_path = "/Users/clairedanicich/Desktop/ADA/wedge/wedge2020/"
service_file = "MSBA-cfc85a1b18d5.json"
gbq_proj_id = "msba-291619"
gbq_dataset_id = "wedge2"

privatekey = service_path + service_file

# Pass in credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# Establish connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [3]:
# Create new database named Wedge_Queries

db = sqlite3.connect("Wedge_Queries.db")
# Place the cursor in the start of the database
cur =db.cursor

In [4]:
# First Query - Sales by date by hour query 

query1 = (
    """SELECT (EXTRACT(date FROM datetime)) AS Date,
    (EXTRACT(hour FROM datetime)) AS Hour,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `msba-291619.wedge2.transArchive*` 
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour"""
)

# Execute queries with `client.query`
results1 = client.query(
    query1,
    location="US",)

# Create output txt file for the first query
with open('Sales_by_Date_by_Hour.txt',"w") as outfile:
    for row in results1: 
        outfile.write(",".join([str(item) for item in row])+"\n")

In [5]:
# Input the results of the first query into the Wedge_Queries database as the table Sales_by_Date_by_Hour
input_file1 = "Sales_by_Date_by_Hour.txt"

# Connect to the new database
db = sqlite3.connect("Wedge_Queries.db") 
cur = db.cursor()

# Remove the table if it already exists
cur.execute('''DROP TABLE IF EXISTS Sales_by_Date_by_Hour''')

# Create the table in the connected database and set up the schema
cur.execute('''CREATE TABLE Sales_by_Date_by_Hour (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# Input the Sales by Date by Hour text data into the established table in the database
with open(input_file1, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Date_by_Hour (Date, Hour, Sales, Transactions, Items)
        VALUES (?,?,?,?,?)''', line) # Make sure that the ? match up with the column labels for value insertion
db.commit()

In [6]:
# Second Query - Owner, Year, Month, Sales, Transactions, and Items
query2 = (
    """SELECT card_no As Owner,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `msba-291619.wedge2.transArchive*` 
    WHERE card_no != 3
    AND department != 0
    AND department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY Owner, Year, Month
    ORDER BY Owner, Year, Month DESC"""
)

# Execute queries with `client.query`
results2 = client.query(
    query2,
    location="US",
)

#Create output .txt file for the first query
with open('Sales_by_Owner_by_Date.txt',"w") as outfile:
    
    for row in results2: 
        outfile.write(",".join([str(item) for item in row])+"\n")

In [7]:
# Input results of the first query into the Wedge_Queries database as the table Sales_by_Owner_by_Date
input_file2 = "Sales_by_Owner_by_Date.txt"

# Connect to the WedgeTask 3 database
db = sqlite3.connect("Wedge_Queries.db") 
cur = db.cursor()

# Remove the table if it already exists
cur.execute('''DROP TABLE IF EXISTS Sales_by_Owner_by_Date''') 

# Create the table in the connected database and set up schema
cur.execute('''CREATE TABLE Sales_by_Owner_by_Date (
    Owner INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# Input the Sales by Owner by Date text data into the established table in the database
with open(input_file2, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Owner_by_Date (Owner, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line) # Make sure that the ? match up with the column labels for value insertion
db.commit()

In [9]:
# Third Query - Product description by year, month w/columns upc, description, dpt #, dpt name, yr, mo, sales, trans, items
query3 = (
    """SELECT Upc AS UPC,
    description AS Product,
    a.department AS Department,
    b.dept_name AS Dept_Name,
    (EXTRACT(year FROM datetime)) AS Year,
    (EXTRACT(month FROM datetime)) AS Month,
    SUM(total) AS Sales,
    COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
    SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `wedge2.transArchive*` a
    LEFT OUTER JOIN `wedge2.Department` b
    ON a.department = b.department
    WHERE card_no != 3
    AND a.department != 0
    AND a.department != 15
    AND trans_status != 'M'
    AND trans_status != 'C'
    AND trans_status != 'J'
    AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status =  'V'
    OR trans_status = 'R')
    GROUP BY UPC, Product, Department, Dept_Name, Year, Month
    ORDER BY UPC, Product, Department, Dept_Name, Year, Month DESC"""
)

# Execute queries with `client.query`
results3 = client.query(
    query3,
    location="US",)

# Output the thrid query results as a text file
with open('Sales_by_Product_by_Date.txt', 'w') as outfile :
    for row in results3: 
        outfile.write(",".join([str(item) for item in row])+"\n")

In [10]:
# Input the results of the first query into the Wedge_Queries database as the table Sales_by_Product_by_Date
input_file3 = "Sales_by_Product_by_Date.txt"

# connect to the Wedge_Queries database
db = sqlite3.connect("Wedge_Queries.db")
cur = db.cursor()

# Remove the table if it already exists
cur.execute('''DROP TABLE IF EXISTS Sales_by_Product_by_Date''') 

# Create the table in the connected database and set up schema
cur.execute('''CREATE TABLE Sales_by_Product_by_Date (
    UPC STRING,
    Description STRING,
    Department FLOAT,
    Dept_Name STRING,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items TEXT)''')

# Input the Sales by Product by Date text data into the established table in the database
with open(input_file3, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Sales_by_Product_by_Date (UPC, Description, Department, Dept_Name, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?,?)''', line)
db.commit()